# Ontwikkeling positieve Covid19 testuitslagen
> RIVM data door de tijd. Update 22 maart

- toc: false
- branch: master
- badges: false
- comments: true
- author: Jesse van Elteren
- categories: []

Dagelijks publiceert het RIVM de inmiddels welbekende testkaart met daarop het aantal mensen dat per gemeente positief is getest op Covid-19.
Het leek me interessant om het verloop per gemeente te bekijken. Daarom heb ik de bronfiles van het RIVM gedownload en hier wat grafieken van gemaakt.
Hieronder zie je drie grafieken:
- Aantal: Dit laat het totaal aantal positieve testcases zien
- Relatief: Dit laat het totaal aantal positieve testcases per 100.000 inwoners zien
- Groei: Dit laat de toename van het aantal gevallen zien van de afgelopen dagen{% fn 1 %} 
Je kan met de slider het verloop van de epidemie volgen{% fn 2 %}
Dag 0 is 3 maart, hiervoor kon ik geen data terugvinden op de site van het RIVM. Gebruik de slider om de ontwikkeling te zien. De laatste dag is gelijk aan de laatste update.

{{ 'Een factor 2 betekent dat het aantal gevallen 2x zo groot is geworden in 4 dagen. Factor '0' Betekent dat het gelijk is gebleven. Ook wordt groei alleen berekend bij >4 cases' | fndetail: 1 }}
{{ 'Een factor 2 betekent dat het aantal gevallen 2x zo groot is geworden in 4 dagen. Factor '0' Betekent dat het gelijk is gebleven. Ook wordt groei alleen berekend bij >4 cases' | fndetail: 1 }}

> Important: Omdat Nederland lang niet alle gevallen test, is een laag aantal geen garantie dat er weinig gevallen zijn! Het aantal werkelijke gevallen ligt waarschijnlijk (veel?) hoger dan het aantal positieve testen. Ook is het testbeleid niet per se hetzelfde in elke gemeente. Pas als we wel veel meer gaan testen kan je er echt conclusies aan verbinden.

In [3]:
# hide
import pandas as pd
from string import punctuation
import altair as alt

df = pd.read_csv('https://github.com/jvanelteren/covid19/raw/master/confirmed_Covid_19_NL.csv',dtype=object)
num_csv = int(max(df['Count']))+1
df[["Aantal", "Aantal per 100000 inwoners",'BevAant']] = df[["Aantal", "Aantal per 100000 inwoners",'BevAant']].apply(pd.to_numeric)

In [4]:
# hide
def clean_df(df):
    df = df.fillna(0)
    df = df.reindex(sorted(df.columns), axis=1)
    df.rename(columns=dict(zip(df.columns,[str(int(c)) if not c[0].isalpha() else c for c in df.columns])),inplace=True)
    return df

df_aantal = df.pivot(index='Gemeente', columns='Count', values='Aantal').reset_index()
df_aantal = clean_df(df_aantal)

df_relative = df.pivot(index='Gemeente', columns='Count', values='Aantal per 100000 inwoners').reset_index()
df_relative = clean_df(df_relative)

In [5]:
# hide
# make a growth df
df_growth = df_aantal
temp_columns = df_growth.columns
def calc_growth(line):
    growth = [round(y/x-1,2) if x>4 else 0 for x,y in zip(line[:-1],line[1:-1])]
    smoothed_growth = [(x1+x2+x3+x4)/3 for x1,x2,x3,x4 in zip(growth,growth[1:],growth[2:],growth[3:])]
    alternative_growth = [round(y/x-1,2) if x>4 else 0 for x,y in zip(line[:-1],line[3:-1])]
    result = [0]*(len(line)-len(alternative_growth)-1) + alternative_growth + [line[-1]]
    return result
df_growth = pd.DataFrame.from_records(df_growth.apply(calc_growth,axis=1))
df_growth.rename(columns=dict(zip(df_growth.columns,temp_columns)),inplace=True)

In [6]:
# hide
# url1 = 'https://cartomap.github.io/nl/rd/gemeente_2020.topojson'
url2 = 'https://cartomap.github.io/nl/wgs84/gemeente_2020.topojson' 
gemeentes = alt.topo_feature(url2, 'gemeente_2020')

columns = [str(day) for day in range(0, num_csv)]
slider = alt.binding_range(min=0, max=num_csv-1, step=1)
select_year = alt.selection_single(name="select", fields=['day'],
                                   bind=slider, init={'day': num_csv-1})
def serve_chart(df, legend, legend_title):
    maximum = max([max(df_growth[c]) for c in columns]) if legend == 'groei' else max(df[str(num_csv-1)])
    chart = alt.Chart(gemeentes).mark_geoshape(
        stroke='black',
        strokeWidth=0.05
    ).transform_lookup(
        lookup='properties.statnaam',
        from_=alt.LookupData(df, 'Gemeente', columns),
        default = '0'
    ).transform_fold(
        columns, as_=['day', legend]
    ).transform_calculate(
        day='parseInt(datum.day)',
        legend='isValid(datum.'+legend+') ? datum.'+legend+' : -1'  
    ).encode(
        tooltip=[alt.Tooltip('properties.statnaam:N', title="Gemeente"), alt.Tooltip(legend+':Q', title=legend)],

        color = alt.condition(
            'datum.'+legend+' > 0',
            alt.Color(legend+':Q', scale=alt.Scale(scheme='spectral', type='symlog',domain=[0,maximum]),sort='descending',
                      legend=alt.Legend(orient='top',title=legend_title,gradientLength=320,tickCount=3, titleLimit=200)),
            alt.value('#dbe9f6')
        )).add_selection(
        select_year
    ).properties(
        width=320,
        height=400
    ).transform_filter(
        select_year)
#     ).configure_view(
#     strokeWidth=0)
    return chart


In [7]:
#hide_input
serve_chart(df_aantal, 'aantal', 'Positieve tests (cumulatief)')

alt.Chart(...)

In [8]:
#hide_input
serve_chart(df_relative, 'relatief','Cases per 100.000 inwoners')

alt.Chart(...)

In [9]:
#hide_input
serve_chart(df_growth, 'groei', 'Groei tov 3 dagen eerder')

alt.Chart(...)

In [10]:
#hide_input
# alt.hconcat(
#     serve_chart(df_aantal, 'absoluut'), serve_chart(df_relative, 'relatief'),serve_chart(df_growth, 'groei')
# ).resolve_scale(
#     color='independent'
# ).configure_view(
#     stroke=None
# )

In [1]:
#hide_input
test = df_aantal.T
test.columns = test.iloc[-1]
# test = test.iloc[:,4:-2]
test = test[:-1]
test = test.reset_index()
def change(col):
    return ''.join([c for c in col if c not in set(punctuation)-set(['-'])])
res = {c:change(c) for c in test.columns if set(punctuation) & set(c)-set(['-'])}
test.rename(columns=res, inplace=True)
test['Datum'] = list(df[['Datum','Count']].groupby('Count').describe()['Datum']['top'])
columns = list(test.columns)
to_remove = ['Count','Datum']
columns = [c for c in columns if len(c)<30 and c not in to_remove]
select_box = alt.binding_select(options=columns, name='Gemeente')
sel = alt.selection_single(fields=['Gemeente'], bind=select_box, init={'Gemeente': 'Amsterdam'})

alt.Chart(test).transform_fold(
    columns,
    as_=['Gemeente', 'value']
).transform_filter(
    sel  
).mark_line().encode(
    x=alt.X('Datum:O', title='Datum'),
    y=alt.X('value:Q', title='Total aantal positieve testen (cumulatief)'),
).add_selection(
    sel
).properties(
        width=300,
        height=400)

NameError: name 'df_aantal' is not defined

> Warning: Je ziet dat sommige lijntjes heel erg stijl omhoog gaan. De enige manier om ze te laten afvlakken is door minder besmettingen. Houdt je dus aan de adviezen. Weinig positieve testresultaten in jouw gemeente wil niet zeggen dat het er niet meer zijn! 

> Note: Ik ben verder geen specialist en heb deze data ook gewoon van het RIVM. D'r kunnen dus ook geen rechten aan worden ontleend, maar voel je vrij om het te gebruiken. De totalen zullen niet helemaal optellen tot de nationele aantallen omdat er ook gevallen zijn waarvan de woonplaats niet bekend is. Heb je tips laat maar weten!

> Tip: Bedankt Fastpages (heel handige manier om Jupyter notebooks te bloggen) en Altair (voor de visualisaties). Super om dit eens te proberen.